In [1]:
using Pkg 
Pkg.add("LinearAlgebra")

using LinearAlgebra

"Numerical `gradient` calculation over 1st matrix axis Y (rows)"
function gradienty(v::AbstractMatrix)
    dv  = diff(v; dims=1)/2
    a   = [dv[[1],:]; dv]
    a .+= [dv; dv[[end],:]]
    a
end

"Numerical `gradient` calculation over 2nd matrix axis X (cols)"
function gradientx(v::AbstractMatrix)
    dv  = diff(v; dims=2)/2
    a   = [dv[:,[1]] dv]
    a .+= [dv dv[:,[end]]]
    a
end

"""
Numerical gradient of a matrix over first 2 axes - Y and X

Output corresponds to matlab `gradient` function.
First output parameter is gradient matrix for 2nd axis - X. Second output is for 1st axis - Y
"""
function gradient(v::AbstractMatrix)
    return gradientx(v), gradienty(v)
end

"""
Calculate dot product over matrices 1st axis Y (rows)

Returns: vector with elements which are dot product of corresponding matrices rows
"""
function doty(a::AbstractMatrix, b::AbstractMatrix)
    [dot(a[i,:], b[i,:]) for i in 1:size(a)[1]]
end

"""
Calculate dot product over matrices 2nd axis X (cols)

Returns: vector with elements which are dot product of corresponding matrices rows
"""
function dotx(a::AbstractMatrix, b::AbstractMatrix)
    [dot(a[:,i], b[:,i]) for i in 1:size(a)[2]]'
end

"""
Calculate cross product over matrices 1st axis Y (rows)

Returns: vector with elements which are cross product of corresponding matrices rows
"""
function crossy(a::AbstractMatrix, b::AbstractMatrix)
    c = [cross(a[i,:], b[i,:]) for i in 1:size(a)[1]]
    Matrix(hcat(c...)')
end

"""
Calculate cross product over matrices 2nd axis X (cols)

Returns: vector with elements which are cross product of corresponding matrices rows
"""
function crossx(a::AbstractMatrix, b::AbstractMatrix)
    c = [cross(a[:,i], b[:,i]) for i in 1:size(a)[2]]'
    Matrix(vcat(c...)')
end

"""
Gaussian, mean, min and max curvatures of a surface

Gaussian and Mean curvatures
`k,h = surfature(x,y,z)`, where x,y,z are 2d arrays of points on the surface.
k and h are the gaussian and mean curvatures, respectively.

`surfature` returns 2 additional arguments: `k,h,pmax,pmin = surfature(x,y,z)`.
pmax and pmin are the minimum and maximum curvatures at each point, respectively.

Function is shamelessly plagiated from matlab one.
See https://www.mathworks.com/matlabcentral/fileexchange/11168-surface-curvature
"""
function surfature(X::AbstractMatrix, Y::AbstractMatrix, Z::AbstractMatrix)

    # First Derivatives
    Xu,Xv = gradient(X)
    Yu,Yv = gradient(Y)
    Zu,Zv = gradient(Z)

    # Second Derivatives
    Xuu,Xuv = gradient(Xu)
    Yuu,Yuv = gradient(Yu)
    Zuu,Zuv = gradient(Zu)

    Xuv,Xvv = gradient(Xv)
    Yuv,Yvv = gradient(Yv)
    Zuv,Zvv = gradient(Zv)

    # Reshape 2D Arrays into Vectors
    Xu = Xu[:];   Yu = Yu[:];   Zu = Zu[:];
    Xv = Xv[:];   Yv = Yv[:];   Zv = Zv[:];
    Xuu = Xuu[:]; Yuu = Yuu[:]; Zuu = Zuu[:];
    Xuv = Xuv[:]; Yuv = Yuv[:]; Zuv = Zuv[:];
    Xvv = Xvv[:]; Yvv = Yvv[:]; Zvv = Zvv[:];

    Xu          =   [Xu Yu Zu]
    Xv          =   [Xv Yv Zv]
    Xuu         =   [Xuu Yuu Zuu]
    Xuv         =   [Xuv Yuv Zuv]
    Xvv         =   [Xvv Yvv Zvv]

    # First fundamental Coeffecients of the surface (E,F,G)
    E           =   doty(Xu,Xu)
    F           =   doty(Xu,Xv)
    G           =   doty(Xv,Xv)

    m           =   crossy(Xu,Xv)
    p           =   sqrt.(doty(m,m))
    n           =   m./[p p p]

    # Second fundamental Coeffecients of the surface (L,M,N)
    L           =   doty(Xuu,n)
    M           =   doty(Xuv,n)
    N           =   doty(Xvv,n)

    s, t = size(Z)

    # Gaussian Curvature
    K = (L .* N - M .^ 2) ./ (E .* G - F .^ 2)
    K = reshape(K,s,t)

    # Mean Curvature
    H = (E .* N + G .* L - 2 .* F .* M) ./ (2 * (E .* G - F .^ 2))
    H = reshape(H,s,t)

    # Principal Curvatures
    Pmax = H + sqrt.(H .^ 2 - K)
    Pmin = H - sqrt.(H .^ 2 - K)

    K, H, Pmax, Pmin
end

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


surfature

https://www.mathworks.com/matlabcentral/fileexchange/11168-surface-curvature

The inputs (X,Y,Z) are 2D arrays corresponding to the surface being analyzed.

In [2]:
X = rand(5,5)
Y = rand(5,5)
Z = rand(5,5)

# gaussian
surface, curvat_mean, curvat_principal_1, curvat_principal_2 = surfature(X,Y,Z)

([-0.7318233495177551 -13.905340222586759 … 1.6409225641833678 -0.1377796000996903; -0.7850043790616301 16.848581785825864 … -14.995726705940697 -1.3205938226987266; … ; -2.980755520671766 -917.3391637525682 … 34.86098843696948 14.094810568655108; -4.486729634815874 2.5057847733950296 … -1.8098794715811741 -0.8495099306061725], [0.1836033894519483 1.7562437189589803 … -1.484058273486071 -0.10850868704986469; -1.5176471320489613 -4.732778819577478 … -37.27741683328525 -0.34700727266247566; … ; 0.2076831946484276 192.13592119336406 … -23.954995416743476 -14.039210703554739; -0.41864438049598024 -2.468946600708133 … 2.9158557835478853 0.2784386413677523], [1.0585511331121047 5.878104005646247 … -0.7347209696508156 0.2782130931293435; 0.23969665801548334 -2.376804802484904 … 0.20059715449750115 0.8534125997069283; … ; 1.9466161519561964 386.6444078374567 … -0.7390350235305441 -0.5112904417350812; 1.7405204469631783 -0.5742401475198784 … 6.127100779189113 1.2412667712086354], [-0.6913443542

## GLMakie.jl (uses OpenGL as a backend) let's you make interactive 2D and 3D plotting 

In [3]:
# Plot the object
Pkg.add("GLMakie")
Pkg.add("Meshes")

using GLMakie
GLMakie.activate!()


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [4]:
function peaks(; n=49)
    x = LinRange(-3, 3, n)
    y = LinRange(-3, 3, n)
    a = 3 * (1 .- x') .^ 2 .* exp.(-(x' .^ 2) .- (y .+ 1) .^ 2)
    b = 10 * (x' / 5 .- x' .^ 3 .- y .^ 5) .* exp.(-x' .^ 2 .- y .^ 2)
    c = 1 / 3 * exp.(-(x' .+ 1) .^ 2 .- y .^ 2)
    return (x, y, a .- b .- c)
end


function plot_peaks_function()
    x, y, z = peaks()
    x2, y2, z2 = peaks(; n=15)
    fig = Figure(resolution=(1200, 400))
    axs = [Axis3(fig[1, i]; aspect=(1, 1, 1)) for i = 1:3]
    hm = surface!(axs[1], x, y, z)
    wireframe!(axs[2], x2, y2, z2)
    contour3d!(axs[3], x, y, z; levels=20)
    Colorbar(fig[1, 4], hm, height=Relative(0.5))
    fig
end
plot_peaks_function()

Figure()

In [ ]:
# by Lazaro Alonso
using GLMakie, Meshes
let
    u = LinRange(0, 1, 50)
    v = LinRange(0, 2π, 50)
    X1 = [u for u in u, v in v]
    Y1 = [(u^4 - u^2) * cos(v) for u in u, v in v]
    Z1 = [(u^4 - u^2) * sin(v) for u in u, v in v]

    fig, ax, pltobj = surface(X1, Y1, Z1; shading = true, ambient = Vec3f(0.65, 0.65, 0.65),
        backlight = 1.0f0, color = sqrt.(X1 .^ 2 .+ Y1 .^ 2 .+ Z1 .^ 2),
        colormap = :viridis, transparency = true,
        figure = (; resolution = (1200, 800), fontsize = 22))
    Colorbar(fig[1, 2], pltobj, height = Relative(0.5))
    colsize!(fig.layout, 1, Aspect(1, 1.0))
    display(fig)
end;